## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

In [2]:
import mne

# tarefa 1
data = mne.read_epochs("../../datasets/avi/multi/mne_data.fif")

Reading c:\Users\vinny\OneDrive\Documentos\UTFPR\Reconhecimento de Padroes\tutoriais\SSVEP\src\multi\..\..\datasets\avi\multi\mne_data.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   15998.05 ms
        0 CTF compensation matrices available


C:\Users\vinny\AppData\Local\Temp\ipykernel_9284\3050485960.py:4: RuntimeWarning: This filename (../../datasets/avi/multi/mne_data.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data = mne.read_epochs("../../datasets/avi/multi/mne_data.fif")


Not setting metadata
80 matching events found
No baseline correction applied
0 projection items activated


In [3]:
# TAREFA 2
threshold = 0.25

# entendimento dos dados
print(data.get_data().shape)
data

(80, 1, 8192)


Number of events,80
Events,6: 326.5: 47: 87.5: 128.2: 209.3: 4
Time range,0.000 – 15.998 s
Baseline,off


In [10]:
# TAREFA 3

import numpy as np

targets = [float(item) for item in data.event_id.keys()]
print("Possíveis alvos:", targets)

hits = 0

for i in range(len(data)):
    dict_y = {str(value): index  for index, value in enumerate(targets)}

    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        sample = (data[i].compute_psd(method='multitaper', fmin=fmin,
                                          fmax=fmax, verbose=False))
        values = np.array((sample)[0][0])
        values_avg = np.average(values)
        # values_avg = np.median(values)
        # values_avg = np.max(values)
        dict_y[str(target)] = values_avg

    # features.append(sample)
    target_frequency = max(dict_y, key=dict_y.get)
    if float(target_frequency) == float(list(data[i].event_id)[0]):
        hits += 1

# X = np.ndarray(features)
# print("Formato dos dados calculados:", X.shape)

Possíveis alvos: [6.0, 6.5, 7.0, 7.5, 8.2, 9.3]


In [11]:
labels = np.load("../../datasets/avi/multi/labels_multi.npy")

acc = 100 * hits / len(labels)
print(f'\nPorcentagem de acerto: {acc:.2f}%')
# hits, len(labels)


Porcentagem de acerto: 80.00%
